In [1]:
!pip install spacy
!python -m spacy download en_core_web_sm


Defaulting to user installation because normal site-packages is not writeable


Python was not found; run without arguments to install from the Microsoft Store, or disable this shortcut from Settings > Manage App Execution Aliases.


In [3]:
print("hello")

hello


In [9]:
!pip install nltk

import nltk


Defaulting to user installation because normal site-packages is not writeable


ModuleNotFoundError: No module named 'nltk'

In [11]:
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    tokens = word_tokenize(text)
    return tokens

# Example usage
text = "Show me all records from the users table where age is greater than 25"
tokens = preprocess_text(text)
print(tokens)


[nltk_data] Downloading package punkt to C:\Users\Anudeep
[nltk_data]     Bura\AppData\Roaming\nltk_data...


['show', 'me', 'all', 'records', 'from', 'the', 'users', 'table', 'where', 'age', 'is', 'greater', 'than', '25']


[nltk_data]   Unzipping tokenizers\punkt.zip.


In [14]:
import spacy

nlp = spacy.load('en_core_web_sm')
text = "Show me all records from the users table where age is greater than 25"

def recognize_entities(text):
    doc = nlp(text)
    entities = [(entity.text, entity.label_) for entity in doc.ents]
    return entities

# Example usage
entities = recognize_entities(text)
print(entities)


[]


In [15]:
import spacy

# Load the small English model
nlp = spacy.load('en_core_web_sm')

# Process a sample text
text = "Show me all records from the users table where age is greater than 25"
doc = nlp(text)

# Print named entities, part-of-speech tags, and dependencies
for token in doc:
    print(f"{token.text}: {token.pos_}, {token.dep_}")

# Print recognized entities
for entity in doc.ents:
    print(f"{entity.text}: {entity.label_}")


Show: VERB, ROOT
me: PRON, dative
all: DET, det
records: NOUN, dobj
from: ADP, prep
the: DET, det
users: NOUN, compound
table: NOUN, pobj
where: SCONJ, advmod
age: NOUN, nsubj
is: AUX, relcl
greater: ADJ, amod
than: ADP, quantmod
25: NUM, npadvmod


In [22]:
import spacy
from spacy.training.example import Example
from spacy.pipeline.textcat import Config
from spacy.lookups import Lookups
from spacy.util import load_config
from spacy.lookups import load_lookups
import spacy_lookups_data

# Load your spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize lookups
lookups = Lookups()
lookups.add_table("lexeme_norm", spacy_lookups_data.get_table("lexeme_norm"))
lookups.add_table("lemma_lookup", spacy_lookups_data.get_table("lemma_lookup"))

# Add the lookups data to the vocabulary of the model
nlp.vocab.lookups.add_lookups(lookups)

# Add the lookups data to the vocabulary of the model
# Add text classifier to the pipeline
config = {"threshold": 0.5}
textcat = nlp.add_pipe("textcat", config=config)

# Add labels to the text classifier
textcat.add_label("SELECT")
textcat.add_label("INSERT")
textcat.add_label("UPDATE")
textcat.add_label("DELETE")

# Prepare training data
train_data = [
    ("Show me all records from the users table where age is greater than 25", {"cats": {"SELECT": 1, "INSERT": 0, "UPDATE": 0, "DELETE": 0}}),
    ("Add a new user with name John and age 30", {"cats": {"SELECT": 0, "INSERT": 1, "UPDATE": 0, "DELETE": 0}}),
    ("Update the age of user with id 1 to 31", {"cats": {"SELECT": 0, "INSERT": 0, "UPDATE": 1, "DELETE": 0}}),
    ("Delete the user with id 1", {"cats": {"SELECT": 0, "INSERT": 0, "UPDATE": 0, "DELETE": 1}}),
]

# Training loop
optimizer = nlp.initialize()
for epoch in range(10):
    losses = {}
    for text, annotations in train_data:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], losses=losses, sgd=optimizer)
    print(f"Epoch {epoch} - Loss: {losses}")

# Save the trained model
nlp.to_disk("intent_classifier")

# Load the model
nlp = spacy.load("intent_classifier")

# Predict the intent of a new query
def predict_intent(text):
    doc = nlp(text)
    intent = max(doc.cats, key=doc.cats.get)
    confidence = doc.cats[intent]
    return intent, confidence

# Example usage
text = "Show me all records from the users table where age is greater than 25"
intent, confidence = predict_intent(text)
print(f"Intent: {intent}, Confidence: {confidence}")


AttributeError: module 'spacy_lookups_data' has no attribute 'get_table'

In [23]:
import re

def extract_query_components(query):
    # Define regex patterns for different query components
    intent_pattern = r'^\s*(SELECT|UPDATE|INSERT INTO|DELETE FROM)\b'
    select_columns_pattern = r'SELECT\s+(.+?)\s+FROM'
    update_columns_pattern = r'UPDATE\s+\w+\s+SET\s+(.+?)\s+(WHERE|$)'
    insert_columns_pattern = r'INSERT INTO\s+\w+\s*\((.+?)\)\s*VALUES'
    delete_conditions_pattern = r'DELETE FROM\s+\w+\s+WHERE\s+(.+?)\s*;?$'
    select_conditions_pattern = r'WHERE\s+(.+?)\s*;?$'
    update_conditions_pattern = r'WHERE\s+(.+?)\s*;?$'

    # Initialize extracted components
    intent = None
    columns = None
    conditions = None

    # Extract intent
    intent_match = re.search(intent_pattern, query, re.IGNORECASE)
    if intent_match:
        intent = intent_match.group(1).upper()
    
    # Extract columns and conditions based on the intent
    if intent == 'SELECT':
        columns_match = re.search(select_columns_pattern, query, re.IGNORECASE)
        conditions_match = re.search(select_conditions_pattern, query, re.IGNORECASE)
    elif intent == 'UPDATE':
        columns_match = re.search(update_columns_pattern, query, re.IGNORECASE)
        conditions_match = re.search(update_conditions_pattern, query, re.IGNORECASE)
    elif intent == 'INSERT INTO':
        columns_match = re.search(insert_columns_pattern, query, re.IGNORECASE)
        conditions_match = None  # INSERT usually doesn't have conditions
    elif intent == 'DELETE FROM':
        columns_match = None  # DELETE usually doesn't specify columns
        conditions_match = re.search(delete_conditions_pattern, query, re.IGNORECASE)

    # Assign matched groups to columns and conditions
    if columns_match:
        columns = columns_match.group(1).split(', ')
    if conditions_match:
        conditions = conditions_match.group(1)

    return {
        'intent': intent,
        'columns': columns,
        'conditions': conditions
    }

# Example usage
queries = [
    "Show me all records from the users table",
    "Update the age of user with id 5 to 35",
    "Add a new user with name 'Alice' and age 28",
    "Delete the user with id 3"
]

for query in queries:
    components = extract_query_components(query)
    print(f"Query: {query}")
    print(f"Intent: {components['intent']}")
    print(f"Columns: {components['columns']}")
    print(f"Conditions: {components['conditions']}")
    print()


UnboundLocalError: cannot access local variable 'columns_match' where it is not associated with a value

In [1]:
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

def extract_and_print_entities(text):
    # Process the text with spaCy
    doc = nlp(text)
    
    # Extract entities
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    
    # Print entities
    if entities:
        print("Entities found in the text:")
        for entity in entities:
            print(f"Text: {entity[0]}, Label: {entity[1]}")
    else:
        print("No entities found in the text.")

# Example text
text = "Add a new student with name 'Alice', age 14, class 9, section A."

# Extract and print entities
extract_and_print_entities(text)


Entities found in the text:
Text: Alice, Label: PERSON
Text: age 14, Label: DATE
Text: 9, Label: CARDINAL


In [69]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
# Sample text
text = "Add a new student with name 'Alice', age 14, grade 9, section 'A'"
def remove_stop_words(text):
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    gg=' '.join(filtered_words)
    gg=gg.replace('id','ID')
    g=gg.replace("'","").replace(",","").replace("  "," ")
    return g
print(remove_stop_words(text))


Add new student name Alice  age 14 grade 9 section 


[nltk_data] Downloading package stopwords to C:\Users\Anudeep
[nltk_data]     Bura\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Anudeep
[nltk_data]     Bura\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [70]:
##using this for themain thinggg!!!
import spacy
from spacy.matcher import Matcher, PhraseMatcher
from spacy.tokens import Span

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize the matcher
matcher = Matcher(nlp.vocab)

# Define the patterns for matching
patterns = [
    {"label": "CLASS", "pattern": [{"LOWER": "class"}, {"IS_DIGIT": True}]},
    {"label": "SECTION", "pattern": [{"LOWER": "section"}, {"IS_ALPHA": True}]},
    {"label": "NAME", "pattern": [{"LOWER": "name"}, {"IS_ALPHA": True}]},
        {"label": "NAMES", "pattern": [{"LOWER": "names"}, {"IS_ALPHA": True}]},
        {"label": "ID", "pattern": [{"LOWER": "id"}, {"IS_DIGIT": True}]},

    {"label": "AGE", "pattern": [{"LOWER": "age"}, {"IS_DIGIT": True}]},
    {"label": "ID", "pattern": [{"LOWER": "id"}, {"IS_DIGIT": True}]},
    {"label": "ENGLISH", "pattern": [{"LOWER": "english"}, {"IS_DIGIT": True},{"IS_ALPHA": True}]},
    {"label": "HINDI", "pattern": [{"LOWER": "hindi"}, {"IS_DIGIT": True}]},
    {"label": "SCIENCE", "pattern": [{"LOWER": "science"}, {"IS_DIGIT": True}]},
    {"label": "GRADE", "pattern": [{"LOWER": "grade"}, {"IS_DIGIT": True}]},


    
]

# Add patterns to the matcher
for pattern in patterns:
    matcher.add(pattern["label"], [pattern["pattern"]])

# Initialize the phrase matcher for known entities
phrase_matcher = PhraseMatcher(nlp.vocab)

# List of known entities (e.g., sections, subjects, common names)
known_entities = [ "english", "hindi", "science"]
patterns = [nlp(text) for text in known_entities]
phrase_matcher.add("KNOWN_ENTITIES", None, *patterns)

def extract_and_print_entities(text):
    # Process the text with spaCy
    doc = nlp(text)
    
    # Extract entities using NER
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    
    # Extract entities using rule-based matching
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        entities.append((span.text, nlp.vocab.strings[match_id]))
    
    # Extract entities using phrase matching
    phrase_matches = phrase_matcher(doc)
    for match_id, start, end in phrase_matches:
        span = doc[start:end]
        entities.append((span.text, "KNOWN_ENTITY"))
    
    # Print entities
    if entities:
        print("Entities found in the text:")
        for entity in entities:
            print(f"Text: {entity[0]}, Label: {entity[1]}")
    else:
        print("No entities found in the text.")

# Example text
text = "Update name to 'Zane' for student with id 29"
gg=remove_stop_words(text)
# Extract and print entities
print(gg)
extract_and_print_entities(gg)


Update name Zane student ID 29
Entities found in the text:
Text: Update, Label: ORG
Text: Zane, Label: PERSON
Text: 29, Label: CARDINAL
Text: name Zane, Label: NAME
Text: ID 29, Label: ID


In [60]:
from transformers import pipeline

# Load a pre-trained NER model
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

# Example text
text = "Add a new student with name 'Alice', age 14, class 9, section A."

# Extract entities using the pre-trained model
entities = ner_pipeline(text)
for entity in entities:
    print(entity)



RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.

In [8]:
# Function to remove empty lines from a text file
def remove_empty_lines(file_path):
    # Read the content of the file
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Filter out empty lines
    cleaned_lines = [line for line in lines if line.strip()]
    
    # Write the cleaned lines back to the file
    with open(file_path, 'w') as file:
        file.writelines(cleaned_lines)

# Specify the input file
input_file = 'dataset.txt'

# Remove empty lines from the specified file
remove_empty_lines(input_file)

print(f"Empty lines have been removed from {input_file}")


Empty lines have been removed from dataset.txt


In [10]:
import csv

# Function to read and parse the text file
def parse_text_file(file_path):
    data = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        current_entry = {}
        for line in lines:
            if line.startswith("Query:"):
                if current_entry:
                    data.append(current_entry)
                current_entry = {"Query": line.strip().split("Query:")[1].strip().strip('"')}
            elif line.startswith("Intent:"):
                current_entry["Intent"] = line.strip().split("Intent:")[1].strip()
            elif line.startswith("Columns:"):
                current_entry["Columns"] = line.strip().split("Columns:")[1].strip()
            elif line.startswith("Conditions:"):
                current_entry["Conditions"] = line.strip().split("Conditions:")[1].strip()
        if current_entry:
            data.append(current_entry)
    return data

# Function to write data to a CSV file
def write_to_csv(data, output_file):
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ["Query", "Intent", "Columns", "Conditions"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in data:
            writer.writerow(row)

# Specify the input and output files
input_file = 'dataset.txt'
output_file = 'output.csv'

# Parse the text file and write to CSV
data = parse_text_file(input_file)
write_to_csv(data, output_file)

print(f"Data has been written to {output_file}")


Data has been written to output.csv


In [11]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    return ' '.join(tokens)

# Example
nl_query = "Add a new student with name 'Alice', age 14, grade 9, section 'A'"
sql_query = "INSERT INTO students (name, age, grade, section) VALUES ('Alice', 14, 9, 'A');"
print(preprocess_text(nl_query))
print(preprocess_text(sql_query))



add a new student with name 'alice ' , age 14 , grade 9 , section ' a '
insert into students ( name , age , grade , section ) values ( 'alice ' , 14 , 9 , ' a ' ) ;


[nltk_data] Downloading package punkt to C:\Users\Anudeep
[nltk_data]     Bura\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

def train_model(train_data):
    # Tokenize the input and output
    input_texts = [example['input'] for example in train_data]
    target_texts = [example['output'] for example in train_data]

    input_encodings = tokenizer(input_texts, truncation=True, padding=True, max_length=512)
    target_encodings = tokenizer(target_texts, truncation=True, padding=True, max_length=512)

    # Convert to tensors and create dataset
    input_ids = input_encodings['input_ids']
    attention_mask = input_encodings['attention_mask']
    labels = target_encodings['input_ids']

    dataset = CustomDataset(input_ids, attention_mask, labels)

    # Train the model
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
    )

    trainer.train()

# Example training data
train_data = [
    {'input': "Add a new student with name 'Alice', age 14, grade 9, section 'A'", 'output': "INSERT INTO students (name, age, grade, section) VALUES ('Alice', 14, 9, 'A');"},
    # Add more training examples
]

train_model(train_data)


ImportError: 
T5Tokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [ ]:
def generate_sql(natural_language_query):
    inputs = tokenizer.encode("translate English to SQL: " + natural_language_query, return_tensors="pt")
    outputs = model.generate(inputs)
    sql_query = tokenizer.decode(outputs[0])
    return sql_query

# Test the model
test_query = "Add a new student with name 'Alice', age 14, grade 9, section 'A'"
print(generate_sql(test_query))


In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()

def rems(text):
    words = nltk.word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    lemmatized_text = ' '.join(lemmatized_words)
    return lemmatized_text
print("Original text:", text)
print("Lemmatized text:", lemmatized_text)


[nltk_data] Downloading package wordnet to C:\Users\Anudeep
[nltk_data]     Bura\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to C:\Users\Anudeep
[nltk_data]     Bura\AppData\Roaming\nltk_data...


Original text: The animals are running in the fields. Students are getting good grades in their classes. We have multiple names in the list.
Lemmatized text: The animal are running in the field . Students are getting good grade in their class . We have multiple name in the list .


In [1]:
import spacy

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Sample text
text = "Select the names and ages of students where age is greater than 14 and section is A"

# Process the text with spaCy
doc = nlp(text)

# Define potential columns and condition keywords
column_keywords = {"name", "age", "class", "section", "english", "hindi", "science"}
condition_keywords = {"older", "younger", "greater", "less", "than", "above", "below", "equals", "equal", "is", "in", "section"}

# Extract potential columns and conditions
columns = []
conditions = []

for token in doc:
    if token.text.lower() in column_keywords:
        columns.append(token.text)
    elif token.text.lower() in condition_keywords or token.ent_type_ in {"CARDINAL", "NUMBER"}:
        conditions.append(token.text)
    elif token.ent_type_ in {"ORG"} and token.head.text.lower() in condition_keywords:
        conditions.append(token.text)

# Join the conditions into a single string
conditions_str = ' '.join(conditions)

# Print the extracted columns and conditions
print("Columns:", columns)
print("Conditions:", conditions_str)


Columns: ['age', 'section']
Conditions: is greater than is


In [2]:
import nltk
from nltk.tokenize import word_tokenize

# Sample text
text = "Show the names and ages of students who are older than 14 and in section A"

# Tokenize the text
tokens = word_tokenize(text)

# Define potential columns and condition keywords
column_keywords = {"name", "age", "class", "section", "english", "hindi", "science"}
condition_keywords = {"older", "younger", "greater", "less", "than", "above", "below", "equals", "equal", "in", "section"}

# Extract potential columns and conditions
columns = []
conditions = []

for token in tokens:
    if token.lower() in column_keywords:
        columns.append(token)
    elif token.lower() in condition_keywords or token.isdigit():
        conditions.append(token)

# Join the conditions into a single string
conditions_str = ' '.join(conditions)

# Print the extracted columns and conditions
print("Columns:", columns)
print("Conditions:", conditions_str)


Columns: ['section']
Conditions: older than 14 in


In [7]:
import spacy

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')
def extract_and_convert_conditions(text):
    # Process the text with spaCy
    doc = nlp(text)
    
    # Define condition keywords and their SQL equivalents
    condition_mapping = {
        "greater": ">",
        "less": "<",
        "older": ">",
        "younger": "<",
        "than": "",
        "is": "=",
    }
    
    # Extract conditions
    conditions = []
    column = None
    comparator = None
    value = None
    
    for token in doc:
        # Check if the token is part of the condition keywords or is a number
        if token.text.lower() in condition_mapping:
            if token.text.lower() in ["greater", "less", "older", "younger"]:
                comparator = condition_mapping[token.text.lower()]
            elif token.text.lower() == "than":
                # Skip the 'than' keyword
                continue
            elif token.text.lower() == "is":
                comparator = "="
        elif token.ent_type_ == "CARDINAL" or token.ent_type_ == "NUMBER":
            value = token.text
        elif token.text.lower() == "age":
            column = "age"
        elif token.text.lower() == "section":
            column = "section"
        # Add more column mappings here if needed
    
    # Construct SQL condition
    if column and comparator and value:
        sql_condition = f"{column} {comparator} {value}"
        conditions.append(sql_condition)
    
    return conditions


In [8]:
text="Show the names and ages of students who are older than 14 and in section A"
conditions = extract_and_convert_conditions(text)
print(f"Text: {text}")
print(f"SQL Conditions: {conditions}\n")

Text: Show the names and ages of students who are older than 14 and in section A
SQL Conditions: []



In [14]:
import spacy

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Function to extract and convert conditions to SQL format
def extract_and_convert_conditions(text):
    # Process the text with spaCy
    doc = nlp(text)
    
    # Define condition keywords and their SQL equivalents
    condition_mapping = {
        "greater": ">",
        "less": "<",
        "older": ">",
        "younger": "<",
        "than": "",
        "is": "=",
        "in": "in",
    }
    
    # Initialize variables
    conditions = []
    column = None
    comparator = None
    value = None
    found_column = False
    
    for token in doc:
        token_text = token.text.lower()
        
        if token_text in condition_mapping:
            if token_text in ["greater", "less", "older", "younger"]:
                comparator = condition_mapping[token_text]
            elif token_text in ["than", "is"]:
                comparator = condition_mapping[token_text]
        elif token.ent_type_ in ["CARDINAL", "NUMBER"]:
            value = token.text
        elif token_text in ["age", "section", "class", "name"]:
            column = token_text
            found_column = True
        elif found_column and comparator and value:
            # Create the SQL condition
            sql_condition = f"{column} {comparator} {value}"
            conditions.append(sql_condition)
            # Reset variables after processing
            column = None
            comparator = None
            value = None
            found_column = False
    
    return conditions

# Sample texts
texts = [
    "Show the names and ages of students who are older than 14 and in section A",
    "Select students where age is greater than 18 and less than 21",
    "Find all students younger than 15",
]

# Extract and convert conditions from sample texts
for text in texts:
    conditions = extract_and_convert_conditions(text)
    print(f"Text: {text}")
    print(f"SQL Conditions: {conditions}\n")


Text: Show the names and ages of students who are older than 14 and in section A
SQL Conditions: []

Text: Select students where age is greater than 18 and less than 21
SQL Conditions: []

Text: Find all students younger than 15
SQL Conditions: []



In [13]:
import nltk
# Sentence preprocessing (replace with your actual NLP pipeline)
sentence = "Show the names and ages of students who are older than 14 and in section A"
tokens = nltk.word_tokenize(sentence)
pos_tags = nltk.pos_tag(tokens)

# Simplified entity extraction (adapt based on your NLP pipeline's output)
entities = []
for i, (token, pos) in enumerate(pos_tags):
    if pos in ["ADJ", "NOUN"] and (i + 1 < len(pos_tags) and pos_tags[i + 1][0] == "PREP"):
        entities.append(" ".join([token, pos_tags[i + 1][0], pos_tags[i + 2][0]]))

print(entities)  # Output: ["age > 14", "section = A"]

[]


In [24]:
import spacy

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Function to extract comparison entities
def extract_comparisons(text):
    # Process the text with spaCy
    doc = nlp(text)
    
    # Define comparison keywords
    comparison_keywords = {"greater than": ">", "less than": "<", "older than": ">", "younger than": "<"}
    
    # Initialize list to store comparisons
    comparisons = []
    
    # Iterate through the tokens and identify comparison keywords
    for token in doc:
        # Check if the token and its following token match any comparison keyword
        if token.text.lower() in comparison_keywords:
            # Check the next token to form the full comparison phrase
            next_token = doc[token.i + 1] if token.i + 1 < len(doc) else None
            comparison_phrase = f"{token.text.lower()} {next_token.text.lower()}" if next_token else token.text.lower()
            
            if comparison_phrase in comparison_keywords:
                comparisons.append(comparison_phrase)
    
    return comparisons

# Sample texts
texts = [
    "Show the names and ages of students who are older than 14 and in section A",
    "Select students where age is greater than 18 and less than 21",
    "Find all students younger than 15",
]

# Extract comparisons from sample texts
for text in texts:
    comparisons = extract_comparisons(text)
    print(f"Text: {text}")
    print(f"Comparisons: {comparisons}\n")


Text: Show the names and ages of students who are older than 14 and in section A
Comparisons: []

Text: Select students where age is greater than 18 and less than 21
Comparisons: []

Text: Find all students younger than 15
Comparisons: []



In [31]:
text="Show the names and ages of students who are older than 14 and in section A"
nlp=spacy.load("en_core_web_sm")
doc=nlp(text)
for ent in doc.ents:
    print(ent.text+"|"+ent.label_+"|"+spacy.explain(ent.label))

In [38]:
import spacy
import re

# Load pre-trained SpaCy model
nlp = spacy.load("en_core_web_sm")

# Define patterns for comparison and keywords
comparison_patterns = {
    'greater_than': r'(greater|older|more)\s*than\s*(\d+)',
    'less_than': r'(less|fewer|lower)\s*than\s*(\d+)',
    'equals': r'(is|are|equals|same)\s*as\s*([A-Za-z0-9]+)',
    'in_section': r'in\s*section\s*([A-Za-z])',
}

def extract_conditions(sentence):
    doc = nlp(sentence)
    conditions = []

    # Extract key phrases and numeric values
    for token in doc:
        if token.pos_ in ['NOUN', 'ADJ'] and token.dep_ in ['attr', 'nsubj']:
            entity = token.text.lower()
            if 'age' in entity or 'marks' in entity:
                for pattern_name, pattern in comparison_patterns.items():
                    match = re.search(pattern, sentence)
                    if match:
                        if pattern_name == 'greater_than':
                            conditions.append(f"{entity} > {match.group(2)}")
                        elif pattern_name == 'less_than':
                            conditions.append(f"{entity} < {match.group(2)}")
                        elif pattern_name == 'equals':
                            conditions.append(f"{entity} = {match.group(2)}")
                        elif pattern_name == 'in_section':
                            conditions.append(f"{entity} = {match.group(1)}")
    
    return ' AND '.join(conditions)

# Example usage
sentence1 = "Show the names and ages of students who are older than 14 and in section A"
sentence2 = "Find students with english marks greater than 45 and age should be less than 20"

print(extract_conditions(sentence1))  # Output: "age > 14 AND section = A"
print(extract_conditions(sentence2))  # Output: "english marks > 45 AND age < 20"


In [39]:
import spacy
import re

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

# Define patterns for extracting conditions
patterns = {
    'greater_than': r'(\d+)\s*(greater|older|more)\s*than\s*(\d+)',
    'less_than': r'(\d+)\s*(less|fewer|lower)\s*than\s*(\d+)',
    'equals': r'is\s*(equal|same)\s*as\s*([\w\d]+)',
    'in_section': r'in\s*section\s*([A-Za-z])'
}

def extract_conditions(sentence):
    doc = nlp(sentence)
    conditions = []
    
    # Extract conditions using regex patterns
    for pattern_name, pattern in patterns.items():
        matches = re.findall(pattern, sentence)
        for match in matches:
            if pattern_name == 'greater_than':
                conditions.append(f"{match[0]} > {match[2]}")
            elif pattern_name == 'less_than':
                conditions.append(f"{match[0]} < {match[2]}")
            elif pattern_name == 'equals':
                conditions.append(f"{match[1]} = {match[2]}")
            elif pattern_name == 'in_section':
                conditions.append(f"section = {match[0]}")
    
    return ' AND '.join(conditions)

# Example usage
sentence = "Show the names and ages of students who are older than 14 and in section A"
print(extract_conditions(sentence))  # Output: "age > 14 AND section = A"


section = A


In [40]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag

# Sample sentence
sentence = "Get the names of students who scored more than 80 in Math"

# Tokenize and POS tagging
tokens = word_tokenize(sentence)
tagged = pos_tag(tokens)

print(tagged)

# Define rules to map POS tags to SQL components (simplified example)
def map_to_sql(tagged_tokens):
    if "scored" in tokens and "more" in tokens:
        return "SELECT name FROM students WHERE math_score > 80"
    # Add more rules based on your dataset and needs
    return ""

sql_query = map_to_sql(tagged)
print(sql_query)


[('Get', 'VB'), ('the', 'DT'), ('names', 'NNS'), ('of', 'IN'), ('students', 'NNS'), ('who', 'WP'), ('scored', 'VBD'), ('more', 'JJR'), ('than', 'IN'), ('80', 'CD'), ('in', 'IN'), ('Math', 'NN')]
SELECT name FROM students WHERE math_score > 80


In [49]:
def preprocess(naturallang):
    pattern = r'(\d+)(st|nd|rd|th)'
    naturallang=re.sub(pattern, r'\1', naturallang)
    naturallang=naturallang.replace("grade","class")
    words = nltk.word_tokenize(naturallang)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    lemmatized_text = ' '.join(lemmatized_words)
    words = word_tokenize(lemmatized_text)
    stop_words = set(stopwords.words('english'))
    stop_words.remove("a")
    filtered_words = [word for word in words if word.lower() not in stop_words]
    gg=' '.join(filtered_words)
    gg=gg.replace('id','ID')
    g=gg.replace("'","").replace(",","").replace("  "," ")
    return g

In [52]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag, ne_chunk

# Ensure that you have the required NLTK resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

# Sample sentence
sentence = "Find record where the score is greater than 80 and the price is less than 100"
# Tokenize and POS tagging
tokens = word_tokenize(sentence)
tagged_tokens = pos_tag(tokens)
print(tagged_tokens)


[('Find', 'NN'), ('record', 'NN'), ('where', 'WRB'), ('the', 'DT'), ('score', 'NN'), ('is', 'VBZ'), ('greater', 'JJR'), ('than', 'IN'), ('80', 'CD'), ('and', 'CC'), ('the', 'DT'), ('price', 'NN'), ('is', 'VBZ'), ('less', 'JJR'), ('than', 'IN'), ('100', 'CD')]


[nltk_data] Downloading package punkt to C:\Users\Anudeep
[nltk_data]     Bura\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Anudeep Bura\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to C:\Users\Anudeep
[nltk_data]     Bura\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to C:\Users\Anudeep
[nltk_data]     Bura\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [53]:
# Define comparison phrases and their variants
comparison_phrases = [
    "greater than", "more than", "less than", "fewer than",
    "equal to", "not equal to", "between", "in"
]

# Define keywords related to comparisons
keywords = ["score", "value", "amount", "price", "quantity"]


In [54]:
import re

def extract_comparison_entities(sentence):
    comparison_entities = []

    # Check for comparison phrases in the sentence
    for phrase in comparison_phrases:
        pattern = r'\b' + re.escape(phrase) + r'\b'
        matches = re.finditer(pattern, sentence, flags=re.IGNORECASE)
        
        for match in matches:
            start = match.start()
            end = match.end()

            # Extract context around the comparison phrase
            context = sentence[max(0, start - 30):min(len(sentence), end + 30)]
            
            # Check for keywords in the context
            for keyword in keywords:
                keyword_pattern = r'\b' + re.escape(keyword) + r'\b'
                if re.search(keyword_pattern, context, flags=re.IGNORECASE):
                    # Highlight the comparison phrase and keyword
                    highlighted_context = re.sub(keyword_pattern, f'**{keyword}**', context)
                    highlighted_context = re.sub(pattern, f'**{phrase}**', highlighted_context)
                    comparison_entities.append(highlighted_context)
                    break

    return comparison_entities

# Extract comparison entities
comparison_entities = extract_comparison_entities(sentence)

# Print results
for entity in comparison_entities:
    print("Comparison Entity:", entity)


Comparison Entity: ind record where the **score** is **greater than** 80 and the price is less than
Comparison Entity: ater than 80 and the **price** is **less than** 100


In [48]:
def chunk_entities(tagged_tokens):
    chunked = ne_chunk(tagged_tokens)
    entities = []
    
    for subtree in chunked:
        if isinstance(subtree, nltk.Tree):
            entity = " ".join(word for word, tag in subtree.leaves())
            entities.append(entity)
    
    return entities

# Chunk named entities
chunked_entities = chunk_entities(tagged_tokens)
print("Named Entities:", chunked_entities)


Named Entities: ['Find']


In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag

# Ensure that you have the required NLTK resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Sample text
text = "Find records where the score is greater than 80 and the price is less than 100. John Doe bought 300 shares of Acme Corp in 2021."

# Tokenize and POS tagging
tokens = word_tokenize(text)
tagged_tokens = pos_tag(tokens)
print("Tokens and POS Tags:", tagged_tokens)


Tokens and POS Tags: [('Find', 'NN'), ('records', 'NNS'), ('where', 'WRB'), ('the', 'DT'), ('score', 'NN'), ('is', 'VBZ'), ('greater', 'JJR'), ('than', 'IN'), ('80', 'CD'), ('and', 'CC'), ('the', 'DT'), ('price', 'NN'), ('is', 'VBZ'), ('less', 'JJR'), ('than', 'IN'), ('100', 'CD'), ('.', '.'), ('John', 'NNP'), ('Doe', 'NNP'), ('bought', 'VBD'), ('300', 'CD'), ('shares', 'NNS'), ('of', 'IN'), ('Acme', 'NNP'), ('Corp', 'NNP'), ('in', 'IN'), ('2021', 'CD'), ('.', '.')]


[nltk_data] Downloading package punkt to C:\Users\Anudeep
[nltk_data]     Bura\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Anudeep Bura\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
import re
import spacy

# Load spaCy's pre-trained model
nlp = spacy.load("en_core_web_sm")

# Process the text
doc = nlp(text)

# Print named entities
for ent in doc.ents:
    print(f"Entity: {ent.text}, Label: {ent.label_}")

# Visualize dependency parsing (requires Jupyter environment)
from spacy import displacy

# Visualize the dependency parsing
displacy.render(doc, style="dep", jupyter=True)

# Define comparison phrases
comparison_phrases = [
    "greater than", "more than", "less than", "fewer than",
    "equal to", "not equal to", "between", "in"
]

# Extract comparison entities
def extract_comparison_entities(text):
    comparison_entities = []
    
    for phrase in comparison_phrases:
        pattern = r'\b' + re.escape(phrase) + r'\b'
        matches = re.finditer(pattern, text, flags=re.IGNORECASE)
        
        for match in matches:
            start = match.start()
            end = match.end()
            
            # Extract context around the comparison phrase
            context = text[max(0, start - 30):min(len(text), end + 30)]
            comparison_entities.append(context.strip())
    
    return comparison_entities

# Extract comparison entities
comparison_entities = extract_comparison_entities(text)
print("Comparison Entities:", comparison_entities)


Entity: less than 100, Label: CARDINAL
Entity: John Doe, Label: PERSON
Entity: 300, Label: CARDINAL
Entity: Acme Corp, Label: ORG
Entity: 2021, Label: DATE


Comparison Entities: ['nd records where the score is greater than 80 and the price is less than', 'ater than 80 and the price is less than 100. John Doe bought 300 shar', 'ought 300 shares of Acme Corp in 2021.']


In [3]:
import spacy
from spacy.matcher import PhraseMatcher

# Load a spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize the PhraseMatcher with the shared vocabulary
phrase_matcher = PhraseMatcher(nlp.vocab)

# List of column names
columns = ["name", "id", "age", "grade", "section", "english", "hindi", "science"]
patterns = [nlp(text) for text in columns]

# Add the patterns to the PhraseMatcher
phrase_matcher.add("COLUMN", None, *patterns)

# Process a text
doc = nlp("Find all students with age greater than 18 and grade A")

# Apply the PhraseMatcher to the doc
column_matches = phrase_matcher(doc)

# Print the matches
for match_id, start, end in column_matches:
    span = doc[start:end]
    print(f"Column found: {span.text}")


Column found: age
Column found: grade


In [7]:
import spacy
from spacy.matcher import Matcher

# Load a spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize the Matcher with the shared vocabulary
matcher = Matcher(nlp.vocab)

# Define patterns for conditions
patterns = [
    [{"LOWER":{"IN":columns}},{"LOWER": {"IN": ["greater", "less", "equal"]}}, {"LOWER": "than"}, {"IS_DIGIT": True}],  # e.g., "greater than 18"
    [{"LOWER":{"IN":columns}},{"LOWER": "equal"}, {"LOWER": "to"}, {"IS_DIGIT": True}],
      [{"LOWER":{"IN":columns}},{"LOWER": "equal"}, {"LOWER": "to"}, {"IS_ALPHA": True}]  # e.g., "equal to 10"
]

# Add patterns to the matcher
matcher.add("CONDITION", patterns)

# Process a text
doc = nlp("Find all students with age greater than 18 and grade equal to A")

# Apply the Matcher to the doc
condition_matches = matcher(doc)

# Print the matches
for match_id, start, end in condition_matches:
    span = doc[start:end]
    print(f"Condition found: {span.text}")


Condition found: age greater than 18
Condition found: grade equal to A


In [8]:
import spacy
from spacy.matcher import PhraseMatcher, Matcher

# Load a spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize the PhraseMatcher and Matcher
phrase_matcher = PhraseMatcher(nlp.vocab)
matcher = Matcher(nlp.vocab)

# List of column names
columns = ["name", "age", "id", "grade", "section", "english", "hindi", "science"]
column_patterns = [nlp(text) for text in columns]
phrase_matcher.add("COLUMN", None, *column_patterns)

# Define patterns for conditions
condition_patterns = [
    [{"IS_ALPHA": True}, {"IS_ASCII": True, "OP": "?"}, {"IS_DIGIT": True}],  # e.g., "id = 123"
    [{"IS_ALPHA": True}, {"IS_ASCII": True, "OP": "?"}, {"IS_DIGIT": True, "OP": "+"}],  # e.g., "age > 20"
    [{"IS_ALPHA": True}, {"IS_ASCII": True, "OP": "?"}, {"LIKE_NUM": True}]  # e.g., "age = 20.5"
]
matcher.add("CONDITION", condition_patterns)

# Process a text
doc = nlp("SELECT name, age FROM students WHERE id = 123 AND age > 20")

# Apply both matchers to the doc
column_matches = phrase_matcher(doc)
condition_matches = matcher(doc)

# Print column matches
for match_id, start, end in column_matches:
    span = doc[start:end]
    print(f"Column Match found: {span.text}")

# Print condition matches
for match_id, start, end in condition_matches:
    span = doc[start:end]
    print(f"Condition Match found: {span.text}")


Column Match found: name
Column Match found: age
Column Match found: id
Column Match found: age
Condition Match found: d = 123
Condition Match found: age > 20


In [1]:
import mysql.connector
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
import spacy
from spacy.matcher import Matcher, PhraseMatcher
from spacy.tokens import Span
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
import re
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()
nltk.download('words')
from nltk.corpus import words
import spacy
from spacy.matcher import PhraseMatcher, Matcher


[nltk_data] Downloading package stopwords to C:\Users\Anudeep
[nltk_data]     Bura\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Anudeep
[nltk_data]     Bura\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Anudeep
[nltk_data]     Bura\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Anudeep
[nltk_data]     Bura\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to C:\Users\Anudeep
[nltk_data]     Bura\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
#need this
def preprocess(naturallang):
    naturallang=naturallang.replace("marks","")
    naturallang=naturallang.replace("are","")
    naturallang=naturallang.replace("to","")
    pattern = r'(\d+)(st|nd|rd|th)'
    naturallang=re.sub(pattern, r'\1', naturallang)
    naturallang=naturallang.replace("grade","class")
    words = nltk.word_tokenize(naturallang)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    lemmatized_text = ' '.join(lemmatized_words)
    # words = word_tokenize(lemmatized_text)
    # stop_words = set(stopwords.words('english'))
    # stop_words.remove("a")
    # filtered_words = [word for word in words if word.lower() not in stop_words]
    # gg=' '.join(filtered_words)
    gg=lemmatized_text
    gg=gg.replace('id','ID')
    # g=gg.replace("'","").replace(",","").replace("  "," ")
    return gg
#need this!!
english_words = set(words.words())
english_words.remove("a")
def is_english_word(word):
    return word.lower() in english_words

In [3]:

df=pd.read_csv("studentdatset.csv")
queries=df["Query"]
intents=df["Intent"]
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(intents)
X_train, X_test, y_train, y_test = train_test_split(queries, y, test_size=0.2, random_state=42)

# Create and train model
model = make_pipeline(TfidfVectorizer(), LogisticRegression())
model.fit(X_train, y_train)

# Predict intent
def predict_intent(query):
    intent_idx = model.predict([query])[0]
    return label_encoder.inverse_transform([intent_idx])[0]


In [4]:

def getentities(text):
# Load a spaCy model
    nlp = spacy.load("en_core_web_sm")

    # Initialize the PhraseMatcher for columns
    phrase_matcher = PhraseMatcher(nlp.vocab)
    columns = ["name", "id", "age", "grade", "section", "english", "hindi", "science"]
    patterns = [nlp(text) for text in columns]
    phrase_matcher.add("COLUMN", None, *patterns)

    # Initialize the Matcher for conditions
    matcher = Matcher(nlp.vocab)
    condition_patterns = [
        [{"LOWER":{"IN":columns}},{"LOWER": {"IN": ["greater", "less", "equal"]}}, {"LOWER": "than"}, {"IS_DIGIT": True}],  # e.g., "greater than 18"
        [{"LOWER":{"IN":columns}},{"LOWER": "equal"}, {"LOWER": "to"}, {"IS_DIGIT": True}],
        [{"LOWER":{"IN":columns}},{"LOWER": "equal"}, {"LOWER": "to"}, {"IS_ALPHA": True}],
        [{"LOWER": {"IN": ["older", "younger", "equal"]}}, {"LOWER": "than"}, {"IS_DIGIT": True}],
        [{"LOWER":'id'},{"IS_DIGIT": True}]
    ]
    dbcolumns=["age","name","section","class","id","english","hindi","science"]
    patterns=[]
    for colm in dbcolumns:
        patterns.append([{"LOWER": colm}, {"IS_ALPHA":True}])
        patterns.append([{"LOWER": colm}, {"IS_DIGIT":True}])
    condition_patterns+=patterns


    matcher.add("CONDITION", condition_patterns)

    # Process a text
    # text="Get names of students whose english marks are greater than 40"

    pretext=preprocess(text)

    doc = nlp(pretext)
    # Apply the PhraseMatcher and Matcher to the doc
    column_matches = phrase_matcher(doc)
    condition_matches = matcher(doc)
    finalcolumns=[]
    # Print the column matches
    for match_id, start, end in column_matches:
        span = doc[start:end]
        finalcolumns.append(span.text)

    finalconditions=[]
    # Print the condition matches
    for match_id, start, end in condition_matches:
        span = doc[start:end]
        word=span.text.split(" ")
        if len(word)==2: 
            if is_english_word(word[1]):
                pass
            else:
                # print(word[1])
                if not word[1].isdigit:
                    word[1]="\'"+word[1]+"\'"
                finalconditions.append("=".join(word))
        else:
            finalconditions.append(span.text)
    for i in finalcolumns:
        for j in finalconditions:
            if i in j:
                finalcolumns.remove(i)
    relations={
        "greater than":">",
        "less than":"<",
        "equal to":"=",
        "is equal to":"=",
        "older than":"age>",
        "younger than":"age<"
    }
    for i in range(0,len(finalconditions)):
        for key, value in relations.items():
            finalconditions[i]=finalconditions[i].replace(key,value)
    return finalcolumns, finalconditions
# print(finalcolumns)
# print(finalconditions)

In [9]:
text="Update grade to 9 for student with id 25"
print(preprocess(text))
intent=predict_intent(text)
print(intent)
finalcolumns,finalconditions=getentities(text)
print(finalcolumns)
print(finalconditions)
print(genquery(text))

Update class 9 for student with ID 25
UPDATE
[]
['class=9', 'ID=25']
UPDATE students SET class=9 WHERE id=25;


In [6]:
def genquery(text):
    query=""
    if intent=="SELECT":
        cols=""
        condi=""
        for i in range(0,len(finalcolumns)-1):
            cols+=i+","
        cols+=finalcolumns[len(finalcolumns)-1]
        for i in range(0,len(finalconditions)-1):
            condi+=i+" and"
        condi+=finalconditions[len(finalconditions)-1]

        
        query="SELECT "+cols+" FROM students"
        if condi!="":
            query+=" WHERE "+condi+";"
    elif intent=="UPDATE":
        colnames=""
        val=""
        for i in range(len(finalconditions)-1):
            val+=finalconditions[i]
        colnames+=finalconditions[len(finalconditions)-1]
        query="UPDATE students SET "+ val+ " WHERE "+ colnames+";"
    elif intent=="DELETE":
        gg=""
        for i in finalconditions:
            i=i.split("=")
            if i[1].isdigit():
                gg+=i[0]+"="+i[1]
            else:
                gg+=i[0]+"="+"\'"+i[1]+"\'"
        query="DELETE FROM students WHERE "+gg+";"

    elif intent=="INSERT":
        colnames=""
        val=""
        for u in range(0,len(finalconditions)):
            if u==len(finalconditions)-1:
                g=finalconditions[u].split("=")
                colnames+=g[0]
                if g[1].isdigit():
                    val+=g[1]+","
                else:
                    val+="\'"+g[1]+"\'"+","
            else:
                g=finalconditions[u].split("=")
                colnames+=g[0]+","
                if g[1].isdigit():
                    val+=g[1]+","
                else:
                    val+="\'"+g[1]+"\'"+","
        query="INSERT INTO students ("+colnames+") VALUES ("+val+");"
    query=query.replace('ID','id')  
    return query
# print(query)

In [ ]:
if not'9'.isdigit():
    print("ggs")
else:
    print("no ggs")

no ggs
